# Control Flow & Conditionals

*📚 Computer Programming II · 👨‍🏫 Dr. Arif Solmaz*


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⏱️ OTURUM ZAMANLAYICI — Bu hücreyi ilk çalıştırın!
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import time as _time, datetime as _dt, json as _json, os as _os
from IPython.display import display, HTML as _HTML

# ── Persistent tracker file (survives kernel restarts in same runtime) ──
_TRACKER_FILE = _os.path.join(
    '/content' if _os.path.isdir('/content') else '/tmp',
    '.cp2_session_Week_02.json'
)

def _load_tracker():
    """Load previous sessions from file."""
    try:
        with open(_TRACKER_FILE, 'r') as f:
            return _json.load(f)
    except (FileNotFoundError, _json.JSONDecodeError, ValueError):
        return {'sessions': [], 'total_heartbeats': 0}

def _save_tracker(data):
    """Persist tracker data to file."""
    try:
        with open(_TRACKER_FILE, 'w') as f:
            _json.dump(data, f)
    except OSError:
        pass

# ── Load any previous session data ──
_tracker = _load_tracker()
_prev_sessions = len(_tracker['sessions'])
_prev_hb = _tracker['total_heartbeats']

# ── Start new session ──
_SESSION_START = _time.time()
_SESSION_START_STR = _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
_HEARTBEATS = [_time.time()]
_CELLS_RUN = [0]
IDLE_THRESHOLD = 300   # 5 min

# Record this session start
_tracker['sessions'].append({
    'start': _SESSION_START_STR,
    'start_ts': _SESSION_START,
    'heartbeats': [_SESSION_START]
})
_save_tracker(_tracker)

def _heartbeat_hook(*args, **kwargs):
    """Record each cell execution — in-memory + file."""
    now = _time.time()
    _HEARTBEATS.append(now)
    _CELLS_RUN[0] += 1
    # Persist every 5 cells to avoid excessive disk IO
    if _CELLS_RUN[0] % 5 == 0:
        try:
            _t = _load_tracker()
            if _t['sessions']:
                _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
                _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
                _t['total_heartbeats'] = _prev_hb + len(_HEARTBEATS)
                _save_tracker(_t)
        except Exception:
            pass

def _calc_active_time(heartbeats=None):
    """Calculate active time from heartbeat list, capping idle gaps."""
    hb = heartbeats or _HEARTBEATS
    if len(hb) < 2: return 0
    active = 0
    for i in range(1, len(hb)):
        gap = hb[i] - hb[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _calc_total_active_time():
    """Calculate total across ALL sessions (multi-day support)."""
    total = 0
    try:
        _t = _load_tracker()
        for sess in _t['sessions'][:-1]:  # all previous sessions
            hb = sess.get('heartbeats', [])
            if len(hb) >= 2:
                for i in range(1, len(hb)):
                    gap = hb[i] - hb[i-1]
                    total += gap if gap <= IDLE_THRESHOLD else 30
    except Exception:
        pass
    # Add current session
    total += _calc_active_time()
    return int(total)

try:
    _ip = get_ipython()
    _ip.events.register('pre_run_cell', _heartbeat_hook)
except Exception: pass

# ── Display ──
_multi = ''
if _prev_sessions > 0:
    _multi = f'<br><span style="font-size:12px;opacity:.75">📂 {_prev_sessions} önceki oturum bulundu — çoklu oturum takibi aktif.</span>'
display(_HTML(f"""<div style='background:linear-gradient(135deg,#667eea,#764ba2);padding:14px 20px;border-radius:10px;color:white;font-family:system-ui;margin:4px 0'>
<b>⏱️ Oturum Başladı</b> — {_SESSION_START_STR}<br>
<span style='font-size:13px;opacity:.85'>Hücre aktiviteniz takip ediliyor. Bitince en alttaki Submit hücresini çalıştırın.</span>{_multi}</div>"""))
print(f'✅ Zamanlayıcı aktif. Idle eşiği: {IDLE_THRESHOLD//60} dk')
if _prev_sessions > 0:
    print(f'📂 Önceki oturumlar: {_prev_sessions} | Toplam heartbeat: {_prev_hb}')

## 🎯 Learning Objectives

- Use comparison operators to compare values
- Understand Boolean values and logical expressions
- Write conditional statements with if, elif, and else
- Combine conditions with logical operators (and, or, not)
- Create nested conditional structures
- Use the ternary conditional expression
- Apply match statement for pattern matching (Python 3.10+)
- Solve engineering problems using conditionals

---
## Part 1: Comparison Operators


Comparison operators are the foundation of decision-making in programming. They compare two values and return a **Boolean** result: either `True` or `False`. Think of them as questions you ask Python about your data.

### Complete List of Comparison Operators

| Operator | Name | Example | Result |
| --- | --- | --- | --- |
| `==` | Equal to | `5 == 5` | `True` |
| `!=` | Not equal to | `5 != 3` | `True` |
| `>` | Greater than | `7 > 3` | `True` |
| `<` | Less than | `3 < 7` | `True` |
| `>=` | Greater or equal | `5 >= 5` | `True` |
| `<=` | Less or equal | `4 <= 5` | `True` |

> 💡 **Note:** **Common Mistake:** Don't confuse `=` (assignment) with `==` (comparison)!

`x = 5` assigns the value 5 to x

`x == 5` checks if x equals 5

#### Basic Comparison Examples

**Figure 1.1: Basic Comparisons**

In [ ]:
# Comparing numbers
a = 10
b = 5

print(f"a = {a}, b = {b}")
print(f"a == b: {a == b}")   # Is a equal to b?
print(f"a != b: {a != b}")   # Is a not equal to b?
print(f"a > b:  {a > b}")    # Is a greater than b?
print(f"a < b:  {a < b}")    # Is a less than b?
print(f"a >= b: {a >= b}")   # Is a greater or equal?
print(f"a <= b: {a <= b}")   # Is a less or equal?

#### Comparing Different Types

**Figure 1.2: Type Comparisons**

In [ ]:
# Comparing integers and floats
print("Integer vs Float:")
print(f"5 == 5.0: {5 == 5.0}")      # True - same value
print(f"5 > 4.9: {5 > 4.9}")        # True
print(f"3.14 < 4: {3.14 < 4}")      # True

# Comparing strings (alphabetical order)
print("\nString comparisons:")
print(f"'apple' < 'banana': {'apple' < 'banana'}")  # True
print(f"'Python' == 'python': {'Python' == 'python'}")  # False!

# String length comparison
name1, name2 = "Alice", "Bob"
print(f"\nlen('{name1}') > len('{name2}'): {len(name1) > len(name2)}")

#### Chained Comparisons

Python's special feature: you can chain comparisons for range checking:

**Figure 1.3: Chained Comparisons**

In [ ]:
# Chained comparisons - Python's elegant feature
age = 25
temperature = 72
score = 85

# Instead of: age >= 18 and age <= 65
print(f"Is {age} between 18 and 65? {18 <= age <= 65}")

# Check if temperature is comfortable (60-80°F)
print(f"Is {temperature}°F comfortable? {60 <= temperature <= 80}")

# Check if score is a 'B' grade (80-89)
print(f"Is {score} a B grade? {80 <= score < 90}")

# Multiple chained comparisons
x = 5
print(f"\nIs 1 < 5 < 10? {1 < x < 10}")
print(f"Is 1 < 5 < 3? {1 < x < 3}")

#### Engineering Application: Tolerance Check

**Figure 1.4: Tolerance Check**

In [ ]:
# Engineering: Check if measurement is within tolerance
target_length = 100.0    # mm
tolerance = 0.5          # ±0.5 mm acceptable
measured = 100.3         # actual measurement

# Calculate acceptable range
lower_limit = target_length - tolerance
upper_limit = target_length + tolerance

# Check if within tolerance using chained comparison
within_tolerance = lower_limit <= measured <= upper_limit

print(f"Target: {target_length} mm (±{tolerance} mm)")
print(f"Acceptable range: {lower_limit} - {upper_limit} mm")
print(f"Measured: {measured} mm")
print(f"Within tolerance: {within_tolerance}")

---
## Part 2: Boolean Values and Expressions


Boolean values represent truth or falsity. In Python, there are exactly two Boolean values: `True` and `False` (note the capitalization). These form the basis of all conditional logic.

### Boolean Type

**Figure 2.1: Boolean Basics**

In [ ]:
# Boolean values
is_raining = True
is_sunny = False

print(f"is_raining: {is_raining} - Type: {type(is_raining)}")
print(f"is_sunny: {is_sunny} - Type: {type(is_sunny)}")

# Booleans from comparisons
temperature = 25
is_hot = temperature > 30
is_cold = temperature < 10
is_comfortable = 15 <= temperature <= 28

print(f"\nTemperature: {temperature}°C")
print(f"Is hot (>30°C): {is_hot}")
print(f"Is cold (<10°C): {is_cold}")
print(f"Is comfortable (15-28°C): {is_comfortable}")

### Truthy and Falsy Values

In Python, values other than `True` and `False` can be evaluated in a Boolean context. Understanding "truthy" and "falsy" values is important:

| Falsy Values (evaluate to False) | Truthy Values (evaluate to True) |
| --- | --- |
| `False` | `True` |
| `None` | Any non-zero number |
| `0`, `0.0` | Non-empty strings |
| Empty string `""` | Non-empty lists, dicts, etc. |
| Empty collections `[]`, `{}`, `()` | Most objects |

**Figure 2.2: Truthy and Falsy**

In [ ]:
# Demonstrating truthy and falsy values
print("Falsy values:")
print(f"bool(0) = {bool(0)}")
print(f"bool(0.0) = {bool(0.0)}")
print(f'bool("") = {bool("")}')
print(f"bool([]) = {bool([])}")
print(f"bool(None) = {bool(None)}")

print("\nTruthy values:")
print(f"bool(1) = {bool(1)}")
print(f"bool(-5) = {bool(-5)}")
print(f'bool("Hello") = {bool("Hello")}')
print(f"bool([1, 2, 3]) = {bool([1, 2, 3])}")
print(f"bool(0.001) = {bool(0.001)}")

---
## Part 3: The if Statement


The `if` statement is the most fundamental control flow structure. It allows your program to execute code only when a certain condition is true.

### Basic Syntax

```
if condition:
    # Code to execute if condition is True
    # (indented with 4 spaces)
```

> 💡 **Note:** **Indentation Matters!** Python uses indentation (typically 4 spaces) to define code blocks. Everything indented under the `if` statement belongs to that block.

#### Simple if Examples

**Figure 3.1: Basic if Statement**

In [ ]:
# Simple if statement
age = 20

if age >= 18:
    print("You are an adult.")
    print("You can vote!")

print("This line always runs.")

**Figure 3.2: Multiple if Statements**

In [ ]:
# Multiple independent if statements
temperature = 28

print(f"Temperature: {temperature}°C")

if temperature > 25:
    print("It's warm outside! 🌞")

if temperature > 30:
    print("It's hot! Stay hydrated! 🥤")

if temperature < 10:
    print("It's cold! Wear a jacket! 🧥")

print("Weather check complete.")

#### Engineering Example: Safety Check

**Figure 3.3: Safety Check**

In [ ]:
# Industrial safety check
pressure = 95  # PSI
max_safe_pressure = 100

print(f"Current Pressure: {pressure} PSI")
print(f"Maximum Safe: {max_safe_pressure} PSI")

if pressure > max_safe_pressure * 0.9:
    print("⚠️ WARNING: Pressure above 90% of maximum!")
    print("Consider reducing load or checking valves.")

if pressure > max_safe_pressure:
    print("🚨 DANGER: Maximum pressure exceeded!")
    print("EMERGENCY SHUTDOWN REQUIRED!")

---
## Part 4: The if-else Statement


The `if-else` statement provides two paths: one when the condition is True, another when it's False. It's like a fork in the road.

### Syntax

```
if condition:
    # Code if condition is True
else:
    # Code if condition is False
```

#### Basic if-else Examples

**Figure 4.1: Voting Eligibility**

In [ ]:
# Check voting eligibility
age = 16

if age >= 18:
    print("You are eligible to vote!")
    print("Make sure you're registered.")
else:
    print("You are not old enough to vote yet.")
    years_left = 18 - age
    print(f"You can vote in {years_left} year(s).")

**Figure 4.2: Even or Odd**

In [ ]:
# Check if number is even or odd
number = 17

if number % 2 == 0:
    print(f"{number} is EVEN")
    print(f"{number} ÷ 2 = {number // 2}")
else:
    print(f"{number} is ODD")
    print(f"Remainder when divided by 2: {number % 2}")

#### Engineering Example: Pass/Fail Test

**Figure 4.3: Quality Control**

In [ ]:
# Quality control test
test_pressure = 145  # PSI
min_required = 150   # Minimum required PSI

print("=== Pressure Test Results ===")
print(f"Measured: {test_pressure} PSI")
print(f"Required: {min_required} PSI")
print()

if test_pressure >= min_required:
    print("✅ TEST PASSED")
    surplus = test_pressure - min_required
    print(f"Margin: +{surplus} PSI")
else:
    print("❌ TEST FAILED")
    deficit = min_required - test_pressure
    print(f"Deficit: -{deficit} PSI")

---
## Part 5: The if-elif-else Chain


When you have multiple conditions to check, use `elif` (else if). This creates a chain where **only one** block executes.

### Syntax

```
if condition1:
    # Code if condition1 is True
elif condition2:
    # Code if condition2 is True
elif condition3:
    # Code if condition3 is True
else:
    # Code if all conditions are False
```

> 💡 **Note:** **Key Point:** Python checks conditions top to bottom and stops at the **first** True condition. Only one block executes.

#### Grade Calculator

**Figure 5.1: Grade Calculator**

In [ ]:
# Grade calculator
score = 85

print(f"Score: {score}")

if score >= 90:
    grade = "A"
    print("Excellent work!")
elif score >= 80:
    grade = "B"
    print("Good job!")
elif score >= 70:
    grade = "C"
    print("Satisfactory.")
elif score >= 60:
    grade = "D"
    print("Needs improvement.")
else:
    grade = "F"
    print("Failed. Please retake.")

print(f"Your grade: {grade}")

#### Sign of a Number

**Figure 5.2: Number Sign**

In [ ]:
# Determine the sign of a number
number = -7

if number > 0:
    print(f"{number} is positive (+)")
elif number < 0:
    print(f"{number} is negative (-)")
else:
    print("The number is zero (0)")

#### BMI Category Calculator

**Figure 5.3: BMI Categories**

In [ ]:
# BMI category calculator
weight = 70  # kg
height = 1.75  # meters

bmi = weight / (height ** 2)
print(f"Weight: {weight} kg")
print(f"Height: {height} m")
print(f"BMI: {bmi:.1f}")
print()

if bmi < 18.5:
    category = "Underweight"
elif bmi < 25:
    category = "Normal weight"
elif bmi < 30:
    category = "Overweight"
else:
    category = "Obese"

print(f"Category: {category}")

#### Engineering: Resistor Color Code

**Figure 5.4: Resistor Colors**

In [ ]:
# Resistor color code lookup
digit = 5

if digit == 0:
    color = "Black"
elif digit == 1:
    color = "Brown"
elif digit == 2:
    color = "Red"
elif digit == 3:
    color = "Orange"
elif digit == 4:
    color = "Yellow"
elif digit == 5:
    color = "Green"
elif digit == 6:
    color = "Blue"
elif digit == 7:
    color = "Violet"
elif digit == 8:
    color = "Gray"
elif digit == 9:
    color = "White"
else:
    color = "Invalid digit"

print(f"Digit {digit} → Color: {color}")

---
## Part 6: Logical Operators


Logical operators allow you to combine multiple conditions. They are essential for creating complex decision logic.

| Operator | Description | Example | Result |
| --- | --- | --- | --- |
| `and` | True if BOTH are True | `True and True` | `True` |
| `or` | True if EITHER is True | `True or False` | `True` |
| `not` | Inverts the value | `not False` | `True` |

#### The AND Operator

**Figure 6.1: AND Operator**

In [ ]:
# AND: Both conditions must be True
age = 25
has_license = True

# Check if person can drive
if age >= 18 and has_license:
    print("✅ You can drive!")
else:
    print("❌ You cannot drive.")

# Truth table for AND
print("\nAND Truth Table:")
print(f"True and True = {True and True}")
print(f"True and False = {True and False}")
print(f"False and True = {False and True}")
print(f"False and False = {False and False}")

#### The OR Operator

**Figure 6.2: OR Operator**

In [ ]:
# OR: At least one condition must be True
is_weekend = True
is_holiday = False

# Check if it's a day off
if is_weekend or is_holiday:
    print("🎉 It's a day off!")
else:
    print("📚 It's a work/school day.")

# Truth table for OR
print("\nOR Truth Table:")
print(f"True or True = {True or True}")
print(f"True or False = {True or False}")
print(f"False or True = {False or True}")
print(f"False or False = {False or False}")

#### The NOT Operator

**Figure 6.3: NOT Operator**

In [ ]:
# NOT: Inverts the Boolean value
is_raining = False

if not is_raining:
    print("☀️ No umbrella needed!")
else:
    print("☔ Take an umbrella!")

# Using NOT with comparisons
temperature = 25

if not (temperature < 0):
    print(f"{temperature}°C is not freezing")

# Double negative
is_closed = True
if not not is_closed:  # Same as: if is_closed
    print("The store is closed")

#### Combining Logical Operators

**Figure 6.4: Complex Conditions**

In [ ]:
# Complex condition: Student discount eligibility
age = 22
is_student = True
has_valid_id = True

# Eligible if: student with valid ID, OR under 18
eligible = (is_student and has_valid_id) or age < 18

print(f"Age: {age}")
print(f"Is Student: {is_student}")
print(f"Has Valid ID: {has_valid_id}")
print(f"Eligible for discount: {eligible}")

# Parentheses matter!
print("\nOperator precedence:")
print(f"True or False and False = {True or False and False}")
print(f"(True or False) and False = {(True or False) and False}")

#### Engineering: Sensor Alarm System

**Figure 6.5: Alarm System**

In [ ]:
# Industrial monitoring: Multiple sensor alarm
temperature = 85   # °C
pressure = 110     # PSI
vibration = 3.5    # mm/s

# Define thresholds
temp_high = temperature > 80
pressure_high = pressure > 100
vibration_high = vibration > 5.0

print("=== Sensor Readings ===")
print(f"Temperature: {temperature}°C (High: {temp_high})")
print(f"Pressure: {pressure} PSI (High: {pressure_high})")
print(f"Vibration: {vibration} mm/s (High: {vibration_high})")
print()

# Alarm conditions
if temp_high and pressure_high:
    print("🚨 CRITICAL: Both temp and pressure high!")
elif temp_high or pressure_high:
    print("⚠️ WARNING: One parameter elevated")
else:
    print("✅ All parameters normal")

---
## Part 7: Nested Conditionals


Nested conditionals are if statements inside other if statements. They allow you to create hierarchical decision trees.

#### Basic Nested Example

**Figure 7.1: Basic Nesting**

In [ ]:
# Nested conditionals: Age and membership check
age = 25
is_member = True

if age >= 18:
    print("You are an adult.")
    if is_member:
        print("Welcome, member! You get a 20% discount.")
    else:
        print("Consider joining our membership program!")
else:
    print("You are a minor.")
    if is_member:
        print("Junior member detected. 10% discount applied.")
    else:
        print("Ask your parents about family membership!")

#### ATM Withdrawal System

**Figure 7.2: ATM System**

In [ ]:
# ATM withdrawal logic
balance = 5000
withdrawal_amount = 1500
daily_limit = 3000
pin_correct = True

print(f"Balance: {balance} TL")
print(f"Requested: {withdrawal_amount} TL")
print()

if pin_correct:
    print("✓ PIN verified")
    if withdrawal_amount <= balance:
        print("✓ Sufficient funds")
        if withdrawal_amount <= daily_limit:
            print("✓ Within daily limit")
            new_balance = balance - withdrawal_amount
            print(f"\n💵 Dispensing {withdrawal_amount} TL")
            print(f"New balance: {new_balance} TL")
        else:
            print(f"❌ Exceeds daily limit of {daily_limit} TL")
    else:
        print("❌ Insufficient funds")
else:
    print("❌ Incorrect PIN")

> 💡 **Note:** **Best Practice:** Avoid deeply nested conditionals (more than 3 levels). They become hard to read. Consider using logical operators or early returns instead.

---
## Part 8: Ternary Conditional Expression


The ternary operator is a compact way to write simple if-else statements in a single line. It's also called the conditional expression.

### Syntax

```
value_if_true if condition else value_if_false
```

#### Basic Examples

**Figure 8.1: Ternary Basics**

In [ ]:
# Simple ternary expression
age = 20
status = "adult" if age >= 18 else "minor"
print(f"Age {age}: {status}")

# Comparing with if-else
# Traditional way:
# if age >= 18:
#     status = "adult"
# else:
#     status = "minor"

# More examples
number = 7
parity = "even" if number % 2 == 0 else "odd"
print(f"{number} is {parity}")

temperature = 25
comfort = "comfortable" if 18 <= temperature <= 26 else "uncomfortable"
print(f"{temperature}°C is {comfort}")

#### Using Ternary with Functions

**Figure 8.2: Ternary with Functions**

In [ ]:
# Ternary in print statements
score = 85
print(f"Result: {'PASS' if score >= 60 else 'FAIL'}")

# Ternary with calculations
x, y = 10, 5
maximum = x if x > y else y
minimum = x if x < y else y
print(f"Max of {x} and {y}: {maximum}")
print(f"Min of {x} and {y}: {minimum}")

# Absolute value using ternary
num = -7
absolute = num if num >= 0 else -num
print(f"Absolute value of {num}: {absolute}")

#### Nested Ternary (Use Sparingly)

**Figure 8.3: Nested Ternary**

In [ ]:
# Nested ternary (less readable - use with caution)
score = 75

# Nested ternary for grades
grade = "A" if score >= 90 else "B" if score >= 80 else "C" if score >= 70 else "F"
print(f"Score {score} → Grade {grade}")

# Better: Sign of a number
num = -5
sign = "positive" if num > 0 else "negative" if num < 0 else "zero"
print(f"{num} is {sign}")

---
## Part 9: Match Statement (Python 3.10+)


The `match` statement (introduced in Python 3.10) provides structural pattern matching, similar to switch/case in other languages but more powerful.

### Basic Syntax

```
match variable:
    case pattern1:
        # Code for pattern1
    case pattern2:
        # Code for pattern2
    case _:
        # Default case (wildcard)
```

#### Simple Match Example

**Figure 9.1: Basic Match**

In [ ]:
# Day of week using match
day = 3

match day:
    case 1:
        print("Monday")
    case 2:
        print("Tuesday")
    case 3:
        print("Wednesday")
    case 4:
        print("Thursday")
    case 5:
        print("Friday")
    case 6 | 7:  # Multiple patterns
        print("Weekend!")
    case _:  # Default case
        print("Invalid day")

#### HTTP Status Codes

**Figure 9.2: HTTP Status**

In [ ]:
# HTTP status code handler
status_code = 404

match status_code:
    case 200:
        message = "OK - Request successful"
    case 201:
        message = "Created - Resource created"
    case 400:
        message = "Bad Request - Invalid syntax"
    case 401:
        message = "Unauthorized - Authentication required"
    case 403:
        message = "Forbidden - Access denied"
    case 404:
        message = "Not Found - Resource doesn't exist"
    case 500:
        message = "Internal Server Error"
    case _:
        message = "Unknown status code"

print(f"Status {status_code}: {message}")

#### Calculator with Match

**Figure 9.3: Calculator**

In [ ]:
# Simple calculator using match
a, b = 15, 4
operation = "/"

match operation:
    case "+":
        result = a + b
        op_name = "Addition"
    case "-":
        result = a - b
        op_name = "Subtraction"
    case "*":
        result = a * b
        op_name = "Multiplication"
    case "/":
        result = a / b if b != 0 else "Error: Division by zero"
        op_name = "Division"
    case "//":
        result = a // b if b != 0 else "Error"
        op_name = "Floor Division"
    case "%":
        result = a % b if b != 0 else "Error"
        op_name = "Modulus"
    case "**":
        result = a ** b
        op_name = "Exponentiation"
    case _:
        result = "Unknown"
        op_name = "Unknown operation"

print(f"{op_name}: {a} {operation} {b} = {result}")

---
## Part 10: Common Conditional Patterns


Here are some frequently used patterns when working with conditionals in real programs.

#### Input Validation

**Figure 10.1: Input Validation**

In [ ]:
# Validate user input
user_input = "25"

# Check if input is a valid positive integer
if user_input.isdigit():
    number = int(user_input)
    if number > 0:
        print(f"✅ Valid positive number: {number}")
    else:
        print("❌ Number must be positive")
else:
    print("❌ Invalid input: not a number")

# Validate email format (simple check)
email = "student@istun.edu.tr"
if "@" in email and "." in email:
    print(f"✅ Email format looks valid: {email}")
else:
    print("❌ Invalid email format")

#### Range Classification

**Figure 10.2: Range Classification**

In [ ]:
# Classify values into ranges
values = [15, 45, 72, 88, 95, 33, 67]

print("Score Classifications:")
for score in values:
    if score >= 90:
        category = "Excellent"
    elif score >= 70:
        category = "Good"
    elif score >= 50:
        category = "Average"
    else:
        category = "Needs Work"
    print(f"  {score}: {category}")

#### Default Values

**Figure 10.3: Default Values**

In [ ]:
# Setting default values
username = ""  # Empty string (could be from user input)
greeting = "Guest"  # Default value

# Method 1: Traditional if-else
if username:
    greeting = username
print(f"Hello, {greeting}!")

# Method 2: Using or operator (Python idiom)
username2 = "Ahmet"
display_name = username2 or "Anonymous"
print(f"Welcome, {display_name}!")

# With None check
config_value = None
actual_value = config_value if config_value is not None else "default"
print(f"Config: {actual_value}")

---
## ❌ Common Mistakes to Avoid


These are the most frequent errors students make with control flow. Study them carefully — recognizing these patterns will save you hours of debugging!

**Using `=` instead of `==`**

`if x = 5:` → SyntaxError! Use `if x == 5:`

                    Remember: `=` assigns a value, `==` compares two values.

**Forgetting the colon `:`**

`if x > 10` → SyntaxError! Every `if`, `elif`, and `else` line must end with a colon `:`

**Incorrect indentation**

                    All code inside an if-block must be indented by the same amount (4 spaces). Mixing tabs and spaces causes `IndentationError`. Configure your editor to use spaces.

**Using `elif` after `else`**

`else` must always be LAST in the chain. You cannot write `elif` after `else` — Python will raise a SyntaxError.

**Comparing different types without conversion**

`if input("Age: ") > 18:` → This compares a string to an int, which always gives unexpected results!

                    Fix: `if int(input("Age: ")) > 18:`

---
# 📝 Exercises


### Exercise 1: Age Category  (Easy)

Given age = 15, print whether the person is a "Child" (0-12), "Teenager" (13-19), or "Adult" (20+).

**Expected Output:**
```
Teenager
```

<details>
<summary>💡 Hints</summary>

- Use if-elif-else chain
- Check age ranges in order
</details>

In [ ]:
# ✏️ [EX1]
age = 15
# Determine age category

### Exercise 2: Even or Odd  (Easy)

Given number = 42, print whether it is "Even" or "Odd".

**Expected Output:**
```
42 is Even
```

<details>
<summary>💡 Hints</summary>

- Use modulus operator: `number % 2`
- If result is 0 → Even, otherwise → Odd
- Use f-string: `f"{number} is Even"`
</details>

In [ ]:
# ✏️ [EX2]
number = 42
# Check if even or odd

### Exercise 3: Grade Calculator  (Easy)

Given score = 78, calculate the letter grade: A (90+), B (80-89), C (70-79), D (60-69), F (below 60).

**Expected Output:**
```
Score: 78 → Grade: C
```

<details>
<summary>💡 Hints</summary>

- Use if-elif-else chain starting from highest grade
- Check `score >= 90` first, then 80, 70, 60
- Use `else` for F (anything below 60)
- Format: `f"Score: {score} → Grade: {grade}"`
</details>

In [ ]:
# ✏️ [EX3]
score = 78
# Calculate letter grade

### Exercise 4: Number Sign  (Easy)

Given num = -5, print whether it's "Positive", "Negative", or "Zero".

**Expected Output:**
```
-5 is Negative
```

<details>
<summary>💡 Hints</summary>

- Three conditions: `num > 0`, `num < 0`, `num == 0`
- Use if-elif-else structure
- Format: `f"{num} is Negative"`
</details>

In [ ]:
# ✏️ [EX4]
num = -5
# Determine sign

### Exercise 5: Leap Year  (Medium)

Check if year = 2024 is a leap year. A year is a leap year if divisible by 4, except century years must be divisible by 400.

**Expected Output:**
```
2024 is a leap year
```

<details>
<summary>💡 Hints</summary>

- Divisible by 400 → leap year
- Divisible by 100 but not 400 → NOT leap year
- Divisible by 4 → leap year
</details>

In [ ]:
# ✏️ [EX5]
year = 2024
# Check if leap year

### Exercise 6: Largest of Three  (Medium)

Find the largest of three numbers: a=15, b=22, c=18.

**Expected Output:**
```
The largest is 22
```

<details>
<summary>💡 Hints</summary>

- Method 1: Use nested if-elif checking all comparisons
- Method 2: Use Python's built-in `max(a, b, c)`
- If using conditionals: check if `a >= b and a >= c`
</details>

In [ ]:
# ✏️ [EX6]
a, b, c = 15, 22, 18
# Find the largest

### Exercise 7: Ticket Price  (Medium)

Calculate ticket price: Children (0-12): 10 TL, Teens (13-17): 15 TL, Adults (18-64): 25 TL, Seniors (65+): 12 TL. Use age = 35.

**Expected Output:**
```
Age 35: Ticket price is 25 TL
```

<details>
<summary>💡 Hints</summary>

- Use if-elif-else chain for age ranges
- Check: `age <= 12`, `age <= 17`, `age <= 64`, else senior
- Store price in a variable, then print result
</details>

In [ ]:
# ✏️ [EX7]
age = 35
# Calculate ticket price

### Exercise 8: Quadrant Finder  (Medium)

Given coordinates x=5, y=-3, determine which quadrant the point is in (or if it's on an axis).

**Expected Output:**
```
Point (5, -3) is in Quadrant IV
```

<details>
<summary>💡 Hints</summary>

- Q1: x>0, y>0 | Q2: x<0, y>0
- Q3: x<0, y<0 | Q4: x>0, y<0
- Check for axes first (x=0 or y=0)
</details>

In [ ]:
# ✏️ [EX8]
x, y = 5, -3
# Determine quadrant

### Exercise 9: Triangle Validity  (Medium)

Check if sides a=3, b=4, c=5 can form a valid triangle (sum of any two sides > third side).

**Expected Output:**
```
Sides 3, 4, 5 form a valid triangle
```

<details>
<summary>💡 Hints</summary>

- Check all three conditions with `and`:
- `a + b > c` and `b + c > a` and `a + c > b`
- If all true → valid triangle
</details>

In [ ]:
# ✏️ [EX9]
a, b, c = 3, 4, 5
# Check triangle validity

### Exercise 10: BMI Calculator  (Medium)

Calculate BMI (weight/height²) for weight=75 kg, height=1.80 m. Classify: Underweight (<18.5), Normal (18.5-24.9), Overweight (25-29.9), Obese (30+).

**Expected Output:**
```
BMI: 23.1 - Normal weight
```

<details>
<summary>💡 Hints</summary>

- BMI formula: `bmi = weight / (height ** 2)`
- Use if-elif-else to classify BMI ranges
- Format to 1 decimal: `f"BMI: {bmi:.1f}"`
</details>

In [ ]:
# ✏️ [EX10]
weight = 75  # kg
height = 1.80  # meters
# Calculate BMI and classify

### Exercise 11: Shipping Cost  (Medium)

Calculate shipping for weight=3.5 kg: 0-1kg: 10 TL, 1-3kg: 20 TL, 3-5kg: 30 TL, 5+kg: 50 TL.

**Expected Output:**
```
Weight: 3.5 kg → Shipping: 30 TL
```

<details>
<summary>💡 Hints</summary>

- Use if-elif-else chain for weight ranges
- Check: `weight <= 1`, `weight <= 3`, `weight <= 5`
- Use `else` for 5+ kg (50 TL)
</details>

In [ ]:
# ✏️ [EX11]
weight = 3.5  # kg
# Calculate shipping cost

### Exercise 12: Password Strength  (Challenge)

Check password strength: Weak (length<6), Medium (6-9), Strong (10+). Test with "MyP@ss123".

**Expected Output:**
```
Password strength: Strong (9 characters)
```

<details>
<summary>💡 Hints</summary>

- Get password length: `len(password)`
- Check length ranges with if-elif-else
- Format: `f"Password strength: {strength} ({length} characters)"`
</details>

In [ ]:
# ✏️ [EX12]
password = "MyP@ss123"
# Check password strength

### Exercise 13: Temperature Alert  (Challenge)

Given temp=85°C and max_temp=80°C: Show "CRITICAL" if >100%, "WARNING" if >90%, "CAUTION" if >80%, else "NORMAL".

**Expected Output:**
```
Temperature: 85°C (106.2% of max)
Status: CRITICAL
```

<details>
<summary>💡 Hints</summary>

- Calculate percentage: `percentage = (temp / max_temp) * 100`
- Check percentage thresholds with if-elif-else
- Format percentage: `f"{percentage:.1f}%"`
</details>

In [ ]:
# ✏️ [EX13]
temp = 85  # Current temperature
max_temp = 80  # Maximum safe temperature
# Determine alert status

### Exercise 14: Ternary Practice  (Easy)

Use a ternary expression to set discount to 20 if is_member is True, otherwise 0. Then print the discount.

**Expected Output:**
```
Discount: 20%
```

<details>
<summary>💡 Hints</summary>

- Ternary syntax: `value_if_true if condition else value_if_false`
- Example: `discount = 20 if is_member else 0`
- Print: `f"Discount: {discount}%"`
</details>

In [ ]:
# ✏️ [EX14]
is_member = True
# Use ternary to set discount

### Exercise 15: Voltage Safety Check  (Challenge)

Check voltage (220V) against range 210-230V. Also check if current (15A) exceeds max (20A). Report overall system status.

**Expected Output:**
```
=== System Status ===
Voltage: 220V (OK)
Current: 15A (OK)
Overall: SYSTEM NORMAL
```

<details>
<summary>💡 Hints</summary>

- Check voltage in range: `min_voltage <= voltage <= max_voltage`
- Check current: `current <= max_current`
- Use ternary for OK/FAULT: `"OK" if condition else "FAULT"`
- Overall normal if both voltage and current are OK
</details>

In [ ]:
# ✏️ [EX15]
voltage = 220  # Volts
current = 15   # Amperes
min_voltage, max_voltage = 210, 230
max_current = 20
# Check system status

### Exercise 🌉: Bridge Exercise: Sneak Peek at Week 3  (Preview)

**Next week you'll learn loops.** For now, write a program that checks if a password is correct. If incorrect, print "Try again!" — but you'd have to copy the code 3 times to allow 3 attempts. Imagine how tedious this would be for 100 attempts!

**Expected Output:**
```
Attempt 1: Wrong password! Try again...
Attempt 2: Wrong password! Try again...
Attempt 3: Access granted!

Imagine doing this 100 times without loops! 😫
Next week: while loops will solve this elegantly!
```

<details>
<summary>💡 Hints</summary>

- Use `if password == correct_password:` for each attempt
- Notice how the same logic is repeated 3 times — that's a signal you need a loop!
- Next week's `while` loop will let you repeat until the password is correct
</details>

In [ ]:
# ✏️ [EXBridge]
# Bridge Exercise: Password Checker WITHOUT Loops
correct_password = "python123"

# Attempt 1
password = "wrong"  # Simulating user input
if password == correct_password:
    print("Attempt 1: Access granted!")
else:
    print("Attempt 1: Wrong password! Try again...")

# Attempt 2 - same code again!
password = "also_wrong"
if password == correct_password:
    print("Attempt 2: Access granted!")
else:
    print("Attempt 2: Wrong password! Try again...")

# Attempt 3 - AGAIN the same code!
password = "python123"
if password == correct_password:
    print("Attempt 3: Access granted!")
else:
    print("Attempt 3: Wrong password! Last chance used.")

# THINK: What if we needed 100 attempts?
print("\nImagine doing this 100 times without loops! 😫")
print("Next week: while loops will solve this elegantly!")

---
# 📮 Submit Your Work

**When you're done with all exercises:**
1. **Run all exercise cells** (make sure each one executed)
2. Fill in your info in the cell below and run it
3. Run the next cell to submit


In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 1: Fill in your info below, then run this cell
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

STUDENT_ID    = ""     # e.g. "2024001234"
STUDENT_NAME  = ""     # e.g. "Ahmet Yılmaz"
STUDENT_EMAIL = ""     # e.g. "ahmet.yilmaz@istun.edu.tr"
CLASS_CODE    = ""     # code given in class

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Don't change anything below this line
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import re as _re

_errors = []
if not _re.match(r"^\d{6,12}$", STUDENT_ID):
    _errors.append("❌ Student ID must be 6-12 digits")
if len(STUDENT_NAME.strip().split()) < 2:
    _errors.append("❌ Enter first and last name")
if not STUDENT_EMAIL.strip().lower().endswith("@istun.edu.tr") or len(STUDENT_EMAIL.strip()) < 16:
    _errors.append("❌ Use your @istun.edu.tr email")
if len(CLASS_CODE.strip()) < 4:
    _errors.append("❌ Invalid class code")

if _errors:
    for _e in _errors:
        print(_e)
    print("\n⚠️  Fix the errors above and run this cell again.")
else:
    print(f"✅ Info OK — {STUDENT_NAME} ({STUDENT_ID})")
    print(f"   {STUDENT_EMAIL}")
    print(f"\n👉 Now run the NEXT cell to submit.")

In [ ]:
_ORIGINAL_HASHES = {
    "ex1": "998f307e1669664278b45e4af500c178",
    "ex2": "1f2bbf41294214b8d49c6eb3f7139c3d",
    "ex3": "7c120c74514b83648d2b9cf2bb2e7642",
    "ex4": "83365511836f3c38f1dd4776d46542f0",
    "ex5": "6f32ad1ec9389213f18786a5d4837d2e",
    "ex6": "2a0b478ad4bc7e1d94de76371ea753f4",
    "ex7": "1457a5d60911b5ae0d376dfa818ff189",
    "ex8": "2928eb2e73f4d3247f007e1ff98784da",
    "ex9": "66271fda52d92efa42d0ae054a8051f2",
    "ex10": "b577e9e5f8de6cb837255e711f99612e",
    "ex11": "8bf5b3abef61a53755b8693fc9fb1dfd",
    "ex12": "366d78f957c9a309653ecf57eb9bf02d",
    "ex13": "6b274d686131a1f20b06cfdb07bfb96e",
    "ex14": "a2303ea41daed8100946f79a054136e8",
    "ex15": "d63f6d6f5cdf31fbe966d07c8c104a1f"
}

#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 📮 STEP 2: Run this cell to submit
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# ⚠️  Make sure you RAN all exercise cells first!

import json, re, os, urllib.request, hashlib
import time as _time, datetime as _dt

WEEK = "Week_02"
URL  = "https://script.google.com/macros/s/AKfycbyf1D3HGSAX4MoIhNlAuWlGrFyyvbM5MIv7ZsLxrVDlATUihrRGEAaibvIZYlCfd8Me/exec"

# ══════════════════════════════════════════════════════════
# SELF-HEALING TIMER — works even if Cell 1 was skipped
# ══════════════════════════════════════════════════════════
_TRACKER_FILE = os.path.join(
    '/content' if os.path.isdir('/content') else '/tmp',
    f'.cp2_session_{WEEK}.json'
)
IDLE_THRESHOLD = 300

def _sh_calc_active(heartbeats):
    """Calculate active seconds from a heartbeat list."""
    if len(heartbeats) < 2: return 0
    active = 0
    for i in range(1, len(heartbeats)):
        gap = heartbeats[i] - heartbeats[i-1]
        active += gap if gap <= IDLE_THRESHOLD else 30
    return int(active)

def _collect_timing():
    """
    Collect timing data from all available sources:
      1. In-memory variables (timer cell ran in this kernel)
      2. Persistent tracker file (previous sessions / kernel restarts)
      3. Minimal fallback (nothing available)
    """
    _info = {
        'active_time': 0, 'wall_time': 0, 'cells_run': 0,
        'heartbeat_count': 0, 'session_count': 0,
        'session_start': '', 'session_end': _dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'source': 'none'
    }

    # ── Source 1: In-memory (timer cell ran this kernel) ──
    _has_memory = False
    try:
        _s = _SESSION_START        # noqa: F821
        _hb = list(_HEARTBEATS)    # noqa: F821
        _cr = _CELLS_RUN[0]        # noqa: F821
        _has_memory = True
        _info['active_time'] = _sh_calc_active(_hb)
        _info['wall_time'] = int(_time.time() - _s)
        _info['cells_run'] = _cr
        _info['heartbeat_count'] = len(_hb)
        _info['session_start'] = _SESSION_START_STR  # noqa: F821
        _info['source'] = 'memory'
    except NameError:
        pass

    # ── Source 2: Persistent file (adds previous sessions) ──
    try:
        with open(_TRACKER_FILE, 'r') as f:
            _t = json.load(f)
        sessions = _t.get('sessions', [])
        _info['session_count'] = len(sessions)

        if not _has_memory and sessions:
            # Timer cell was NOT run — reconstruct from file
            last = sessions[-1]
            hb = last.get('heartbeats', [])
            _info['active_time'] = _sh_calc_active(hb)
            _info['wall_time'] = int(hb[-1] - hb[0]) if len(hb) >= 2 else 0
            _info['cells_run'] = last.get('cells_run', len(hb))
            _info['heartbeat_count'] = len(hb)
            _info['session_start'] = last.get('start', '')
            _info['source'] = 'file-current'

        # Add previous sessions' active time
        if len(sessions) > 1:
            prev_active = 0
            prev_cells = 0
            prev_hb = 0
            cutoff = -1 if _has_memory else -1  # exclude current session
            for sess in sessions[:cutoff]:
                hb = sess.get('heartbeats', [])
                prev_active += _sh_calc_active(hb)
                prev_cells += sess.get('cells_run', len(hb))
                prev_hb += len(hb)
            _info['prev_active_time'] = prev_active
            _info['prev_cells_run'] = prev_cells
            _info['prev_heartbeats'] = prev_hb
            _info['total_active_time'] = _info['active_time'] + prev_active
            _info['total_cells_run'] = _info['cells_run'] + prev_cells
            if _info['source'] == 'none':
                _info['source'] = 'file-prev'

    except (FileNotFoundError, json.JSONDecodeError, KeyError, ValueError):
        pass

    # Ensure totals exist
    _info.setdefault('total_active_time', _info['active_time'])
    _info.setdefault('total_cells_run', _info['cells_run'])
    return _info

# ── Persist final heartbeat snapshot before collecting ──
try:
    _t = json.load(open(_TRACKER_FILE))
    if _t['sessions']:
        _t['sessions'][-1]['heartbeats'] = list(_HEARTBEATS)
        _t['sessions'][-1]['cells_run'] = _CELLS_RUN[0]
        json.dump(_t, open(_TRACKER_FILE, 'w'))
except Exception:
    pass

_timing = _collect_timing()

# ── Display timing ──
if _timing['source'] == 'none':
    print("⚠️  Zamanlayıcı verisi bulunamadı — ilk hücreyi çalıştırmayı unutmayın.")
    print("    Gönderim yine de yapılacak (süre: 0 olarak kaydedilir).")
else:
    _a = _timing['active_time']
    _w = _timing['wall_time']
    print(f"⏱️  Bu oturum — Aktif: {_a//60}m {_a%60}s  |  Duvar: {_w//60}m {_w%60}s")
    print(f"🔢  Hücreler: {_timing['cells_run']}  |  Heartbeat: {_timing['heartbeat_count']}")
    if _timing['session_count'] > 1:
        _ta = _timing['total_active_time']
        print(f"📂  Toplam {_timing['session_count']} oturum — Toplam aktif: {_ta//60}m {_ta%60}s  |  Toplam hücre: {_timing['total_cells_run']}")
    if _timing['total_active_time'] < 120:
        print("⚠️  Toplam aktif süre < 2 dk — çalıştırdığınız hücre sayısı düşük olabilir.")

_time_on_page = _timing['total_active_time']

# ══════════════════════════════════════════════════════════
# STUDENT INFO CHECK
# ══════════════════════════════════════════════════════════
try:
    _sid = STUDENT_ID.strip()
    _sname = STUDENT_NAME.strip()
    _semail = STUDENT_EMAIL.strip().lower()
    _scode = CLASS_CODE.strip().upper()
except NameError:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın (öğrenci bilgileri).")

if not _sid or not _sname or not _semail or not _scode:
    raise SystemExit("❌ Üstteki hücreyi önce çalıştırın — bazı alanlar boş.")

# ══════════════════════════════════════════════════════════
# EXERCISE ANSWER COLLECTION (improved with retry)
# ══════════════════════════════════════════════════════════
_EX_PATTERN = re.compile(r'#\s*✏️\s*\[EX(\d+)\]')

def _extract_exercise(src):
    """Try to extract exercise ID and code from a source string."""
    m = _EX_PATTERN.match(src)
    if not m: return None, None
    ex_id = 'ex' + m.group(1)
    clean = '\n'.join(src.split('\n')[1:]).strip()
    return ex_id, clean

_answers = {}

# ── Method 1: In[] list (most reliable for Run-All) ──
try:
    for _src in In:
        if not _src: continue
        _eid, _code = _extract_exercise(_src)
        if _eid:
            _answers[_eid] = {
                'code': _code,
                'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
            }
except NameError:
    pass

# ── Method 2: IPython history_manager (flush first!) ──
if not _answers:
    try:
        _ipy = get_ipython()
        # Force flush so Run-All history is fully written
        try: _ipy.history_manager.db.commit()
        except Exception: pass
        _time.sleep(0.5)  # small delay for DB sync
        for _sess, _line, _src in _ipy.history_manager.get_range(output=False):
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                }
    except Exception:
        pass

# ── Method 3: Read notebook file (VS Code / local Jupyter) ──
if not _answers:
    _nb_path = None
    try:
        _nb_path = __vsc_ipynb_file__
    except NameError:
        _candidates = [f for f in os.listdir('.') if f.endswith('.ipynb') and WEEK in f]
        if len(_candidates) == 1:
            _nb_path = _candidates[0]
    if _nb_path and os.path.exists(str(_nb_path)):
        with open(str(_nb_path), 'r', encoding='utf-8') as _f:
            _nb = json.load(_f)
        for _cell in _nb['cells']:
            if _cell['cell_type'] != 'code': continue
            _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
            _eid, _code = _extract_exercise(_src)
            if _eid:
                _answers[_eid] = {
                    'code': _code,
                    'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                }

# ── Method 4: Colab notebook file at /content/ ──
if not _answers:
    try:
        _colab_candidates = [f for f in os.listdir('/content') if f.endswith('.ipynb') and WEEK in f]
        if _colab_candidates:
            _nb_path = os.path.join('/content', _colab_candidates[0])
            with open(_nb_path, 'r', encoding='utf-8') as _f:
                _nb = json.load(_f)
            for _cell in _nb['cells']:
                if _cell['cell_type'] != 'code': continue
                _src = ''.join(_cell['source']) if isinstance(_cell['source'], list) else _cell['source']
                _eid, _code = _extract_exercise(_src)
                if _eid:
                    _answers[_eid] = {
                        'code': _code,
                        'modified': hashlib.md5(_code.encode()).hexdigest() != _ORIGINAL_HASHES.get(_eid, '')
                    }
    except Exception:
        pass

print(f"📝 Found {len(_answers)} exercise(s): {', '.join(sorted(_answers.keys(), key=lambda x: int(x[2:]) if x[2:].isdigit() else 999))}")

if not _answers:
    print("\n⚠️  Hiçbir exercise yanıtı bulunamadı!")
    print("Submit'ten ÖNCE tüm exercise hücrelerini çalıştırdığınızdan emin olun.")
    print("\n💡 İpucu: 'Run All' yerine exercise hücrelerini tek tek çalıştırıp")
    print("   ardından bu hücreyi çalıştırmayı deneyin.")
    raise SystemExit()

# ══════════════════════════════════════════════════════════
# SEND SUBMISSION
# ══════════════════════════════════════════════════════════
_data = json.dumps({
    'week': WEEK,
    'studentId': _sid,
    'studentName': _sname,
    'studentEmail': _semail,
    'classCode': _scode,
    'source': 'cp2-notebook',
    'timeOnPage': _time_on_page,
    'wallTime': _timing['wall_time'],
    'cellsRun': _timing['total_cells_run'],
    'sessionCount': _timing['session_count'],
    'sessionStart': _timing['session_start'],
    'sessionEnd': _timing['session_end'],
    'timerSource': _timing['source'],
    'answers': _answers
}).encode('utf-8')

print("📡 Gönderiliyor...")

try:
    _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
    _resp = urllib.request.urlopen(_req, timeout=30)
    _result = json.loads(_resp.read().decode())
    if _result.get('success'):
        print(f"\n✅ {_result['message']}")
        print('📧 Onay için e-postanızı kontrol edin.')
    else:
        print(f"\n❌ {_result.get('message', 'Gönderim başarısız')}")
except Exception as _e:
    try:
        _req = urllib.request.Request(URL, data=_data, headers={'Content-Type': 'text/plain'}, method='POST')
        urllib.request.urlopen(_req, timeout=10)
    except:
        pass
    print(f"\n⚠️  İstek gönderildi — onay için e-postanızı kontrol edin.")
    print(f"(E-posta gelmezse tekrar deneyin veya hocaya başvurun)")
